In [1]:
import os

In [2]:
%pwd

'd:\\SEM - 4\\Projects\\End-to-End-ML-Project-with-MLflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\SEM - 4\\Projects\\End-to-End-ML-Project-with-MLflow'

In [ ]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/gokhale.g/End-to-End-ML-Project-with-MLflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="gokhale.g"
os.environ["MLFLOW_TRACKING_PASSWORD"]="ec7b06b632047ad710fa071d41155a8f310e50d4"

In [21]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [22]:
from MLflowProject.constants import *
from MLflowProject.utils.common import read_yaml, create_directories, save_json

In [23]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/gokhale.g/End-to-End-ML-Project-with-MLflow.mlflow",
           
        )

        return model_evaluation_config


In [24]:
pip install mlflow

Note: you may need to restart the kernel to use updated packages.


In [25]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [26]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # ⚡ Handle model logging based on backend
            if "dagshub.com" in mlflow.get_tracking_uri():
                # DagsHub: skip registry, just log model
                mlflow.sklearn.log_model(model, "model")
            elif tracking_url_type_store != "file":
                # Custom MLflow server (with DB backend): allow registry
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                # Local file store
                mlflow.sklearn.log_model(model, "model")


In [27]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-08-21 12:10:08,693: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-21 12:10:08,727: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-21 12:10:08,741: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-21 12:10:08,745: INFO: common: created directory at: artifacts]
[2025-08-21 12:10:08,749: INFO: common: created directory at: artifacts/model_evaluation]
[2025-08-21 12:10:09,354: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/08/21 12:10:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/08/21 12:10:20 INFO mlflow.tracking._tracking_service.client: 🏃 View run grandiose-robin-357 at: https://dagshub.com/gokhale.g/End-to-End-ML-Project-with-MLflow.mlflow/#/experiments/0/runs/99c4075dcd8140f7bc0de9f44f2cc2b0.
2025/08/21 12:10:20 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/gokhale.g/End-to-End-ML-Project-with-MLflow.mlflow/#/experiments/0.
